In [1]:
from datetime import date, timedelta
import time

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [2]:
df_train = pd.read_csv(
    '../input/train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    "../input/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [3]:
items = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")

df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train

In [4]:
oil_price = pd.read_csv('../input/oil.csv')
oil_price = oil_price[oil_price.date>='2017-01-01']

In [5]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

In [6]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

items = items.reindex(df_2017.index.get_level_values(1))

In [8]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [9]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "std_3_2017": get_timespan(df_2017, t2017, 3, 3).std(axis=1).values, 
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "std_7_2017": get_timespan(df_2017, t2017, 7, 7).std(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "std_14_2017": get_timespan(df_2017, t2017, 14, 14).std(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "std_30_2017": get_timespan(df_2017, t2017, 30, 30).std(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "std_60_2017": get_timespan(df_2017, t2017, 60, 60).std(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        "std_140_2017": get_timespan(df_2017, t2017, 140, 140).std(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values
    })
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['std_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').std(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
        X['std_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').std(axis=1).values
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [10]:
print("Preparing dataset...")
t2017 = date(2017, 5, 31)
X_l, y_l = [], []
for i in range(6):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

Preparing dataset...


In [11]:
print("Training and predicting models...")

param = {}
param['objective'] = 'reg:linear'
param['eta'] = 0.01
param['max_depth'] = 4
param['silent'] = 1
param['eval_metric'] = 'rmse'
param['min_child_weight'] = 4
param['subsample'] = 0.8
param['colsample_bytree'] = 0.7
param['seed'] = 137
num_rounds = 30000



plst = list(param.items())

MAX_ROUNDS = 30000
val_pred = []
test_pred = []
cate_vars = []

Training and predicting models...


In [12]:
dtest = xgb.DMatrix(X_test)
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = xgb.DMatrix(
        X_train, label=y_train[:, i],
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    dval = xgb.DMatrix(
        X_val, label=y_val[:, i],
        weight=items["perishable"] * 0.25 + 1)
        
    watchlist = [ (dtrain,'train'), (dval, 'val') ]
    model = xgb.train(plst, dtrain, num_rounds, watchlist, early_stopping_rounds=100, verbose_eval=50)
    
    val_pred.append(model.predict(dval, ntree_limit = model.best_ntree_limit))
    test_pred.append(model.predict(dtest, ntree_limit = model.best_ntree_limit))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('xgb_2017_01_06_21_13.csv', float_format='%.4f', index=None)

Step 1
[0]	train-rmse:1.18284	val-rmse:1.1383
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[50]	train-rmse:0.858393	val-rmse:0.824593
[100]	train-rmse:0.696388	val-rmse:0.670996
[150]	train-rmse:0.621931	val-rmse:0.602742
[200]	train-rmse:0.589083	val-rmse:0.574081
[250]	train-rmse:0.574202	val-rmse:0.561843
[300]	train-rmse:0.566731	val-rmse:0.555815
[350]	train-rmse:0.562379	val-rmse:0.552234
[400]	train-rmse:0.559622	val-rmse:0.549909
[450]	train-rmse:0.557847	val-rmse:0.548385
[500]	train-rmse:0.556621	val-rmse:0.547284
[550]	train-rmse:0.55576	val-rmse:0.546523
[600]	train-rmse:0.555139	val-rmse:0.545982
[650]	train-rmse:0.554639	val-rmse:0.54558
[700]	train-rmse:0.554166	val-rmse:0.545184
[750]	train-rmse:0.55378	val-rmse:0.544896
[800]	train-rmse:0.553391	val-rmse:0.544593
[850]	train-rmse:0.55304	val-rmse:0.54433
[900]	train-rmse:0.552754	val-rmse:0.544144
[950]	train-rmse:0.552456	